In [1]:
import os
from numpy import *

glib_path = os.path.abspath("../datapacks/Glib")
data_path = f"{glib_path}/data"
module_path = data_path + "/{m}/functions"

# `glib.block` & `Glib.item` search trees


In [ ]:
os.system(f"python ./block_converter/main.py")

# `_` functions

Generate "_" functions that allow to determine which modules are active or not. They also allow to fix the autocompletion for the user.

In [3]:
exclude_module = ["glib", "glib.core", "minecraft"]

# Generating all "_" functions
for module in os.listdir(data_path):
    if module in exclude_module: continue
    module_type, module_name = module.split(".")
    if not os.path.isdir(module_path.format(m=module)): os.makedirs(module_path.format(m=module))

    with open(f"{module_path.format(m=module)}/_.mcfunction","w") as f:
        msg = {"text":f"[{module} documentation]","color":"dark_aqua","clickEvent":{"action":"open_url","value":f"https://glibs.rtfd.io/en/latest/{module_name}.html"},"hoverEvent":{"action":"show_text","contents":"Click to open URL"}}
        f.write(f"tellraw @s [{msg}]\n".replace("'",'"'))
        f.write(f"scoreboard players set {module} glib.activeModule 1")

# Generating glib.core:load.mcfunction
with open(f"{data_path}/glib.core/functions/load.mcfunction", "r") as old, open(f"{data_path}/glib.core/functions/load_tmp.mcfunction", "w") as new:
    for line in old:
        new.write(line)
        if line.startswith("# Module list"): break
    for module in os.listdir(f"{glib_path}/data"):
        if module in exclude_module: continue
        new.write(f"function {module}:_\n")
os.remove(f"{data_path}/glib.core/functions/load.mcfunction")
os.rename(f"{data_path}/glib.core/functions/load_tmp.mcfunction", f"{data_path}/glib.core/functions/load.mcfunction")

# Menu > Debug

Allow to add/remove tags to debug systems

In [9]:
import libs.menu as menu

group_tag = "glib.menu.active"
menus = []

###############
# Menu / Main #
###############
              
menus.append(("glib.menu","glib.core:menu/main"))

items = []
items.append(("Debug",              "submenu",  "glib.menu.debug"               ))
items.append((" ",                  "text"                                      ))
items.append(("Documentation",      "link",     "https://glibs.rtfd.io"         ))
items.append((" ",                  "text"                                      ))
items.append(("Official website",   "link",     "https://glib.gunivers.net"     ))
items.append((" ",                  "text"                                      ))
items.append(("Our Discord",        "link",     "https://discord.gg/E8qq6tN"    ))
items.append((" ",                  "text"                                      ))
items.append(("Support us",         "link",     "https://discord.gg/E8qq6tN"    ))

menu.paged_menu(
    dest = f"{data_path}/glib.core/functions/menu/main.mcfunction",
    title="Glib Menu",
    menu_tag = "glib.menu",
    group_tag = group_tag,
    items = items,
    exit_msg = str([{"text":"\n"*18},
                        {"text":" Thank you for using Glibs!\n","color":"dark_aqua","bold":True},
                        {"text":" Share us your creations on twitter! ","color":"gray"},
                        {"text":"@Gunivers_\n","color":"gold",
                            "clickEvent":{"action":"open_url","value":"https://twitter.com/Gunivers_"},
                            "hoverEvent":{"action":"show_text","contents":"Visit our Twitter page"}}
                    ]).replace("'",'"').replace('True','true')
)

################
# Menu / Debug #
################

menus.append(("glib.menu.debug","glib.core:menu/debug"))

exclude_module = ["glib", "glib.core", "minecraft"]

items = []
items.append(("Debug stick","tag","glib.debug.stick"))
items.append((" ","text"))

for module_full in os.listdir(data_path):
    if module_full in exclude_module: continue
    module = module_full.split(".")[1]
    items.append((module, "submenu", f"glib.menu.debug.{module}"))

    menus.append((f"glib.menu.debug.{module}",f"glib.core:menu/debug/{module}"))
    module_items = []
    for item in os.listdir(module_path.format(m=module_full)):
        path_item = f"{module_path.format(m=module_full)}/{item}"
        if (os.path.isdir(path_item) and os.path.isfile(f"{path_item}.mcfunction"))\
            or (os.path.isdir(path_item) and os.path.isfile(f"{path_item}_ata.mcfunction"))\
            or (os.path.isdir(path_item) and os.path.isfile(f"{path_item}_tti.mcfunction"))\
            or item in ["_.mcfunction", "child", "accuracy", "config", "debug", "global"]: continue
        
        if os.path.isfile(path_item): module_items.append((item.replace(".mcfunction",""), "tag", f"glib.debug.{module}.{item.replace('.mcfunction','')}"))
        if os.path.isdir(path_item):
            module_items.append((item, "submenu", f"glib.menu.debug.{module}.{item}"))

            menus.append((f"glib.menu.debug.{module}.{item}",f"glib.core:menu/debug/{module}/{item}"))
            submodule_items = []
            for subitem in os.listdir(path_item):
                if os.path.isfile(f"{path_item}/{subitem}") and subitem != "_.mcfunction":
                    submodule_items.append((subitem.replace(".mcfunction",""), "tag", f"glib.debug.{module}.{item}.{subitem.replace('.mcfunction','')}"))

            menu.paged_menu(
                dest = f"{data_path}/glib.core/functions/menu/debug/{module}/{item}.mcfunction",
                title=f"Glib Menu / Debug / {module} / {item}",
                menu_tag = f"glib.menu.debug.{module}.{item}",
                parent_menu_tag = f"glib.menu.debug.{module}",
                group_tag = group_tag,
                items = submodule_items
            )
            

    menu.paged_menu(
        dest = f"{data_path}/glib.core/functions/menu/debug/{module}.mcfunction",
        title=f"Glib Menu / Debug / {module}",
        menu_tag = f"glib.menu.debug.{module}",
        parent_menu_tag = "glib.menu.debug",
        group_tag = group_tag,
        items = module_items
    )

menu.paged_menu(
    dest = f"{data_path}/glib.core/functions/menu/debug.mcfunction",
    title="Glib Menu / Debug",
    menu_tag = "glib.menu.debug",
    parent_menu_tag = "glib.menu",
    group_tag = group_tag,
    items = items
)




dest = f"{data_path}/glib.core/functions/menu.mcfunction"
menu.build(menus, dest = dest)

namespace = dest.split("data/")[1].split("/")[0]
function = dest.split("functions/")[1].replace(".mcfunction","")
print(f'Menus builded successfully!\nNow, put the following command in a loop:\nexecute if entity @a[tag={group_tag}] run function {namespace}:{function}')

Menus builded successfully!
Now, put the following command in a loop:
execute if entity @a[tag=glib.menu.active] run function glib.core:menu
